## Segmentation and Cluster of Neighborhoods in Toronto

Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown at <a href="https://d3c33hcgiwev3.cloudfront.net/imageAssetProxy.v1/7JXaz3NNEeiMwApe4i-fLg_40e690ae0e927abda2d4bde7d94ed133_Screen-Shot-2018-06-18-at-7.17.57-PM.png?expiry=1588809600000&hmac=K2IKamXvUjuNmLLVhDi_Wu2GdwvfoENh4B5fk0IWC6U">the coursera site.</a>
https://d3c33hcgiwev3.cloudfront.net/imageAssetProxy.v1/7JXaz3NNEeiMwApe4i-fLg_40e690ae0e927abda2d4bde7d94ed133_Screen-Shot-2018-06-18-at-7.17.57-PM.png?expiry=1588809600000&hmac=K2IKamXvUjuNmLLVhDi_Wu2GdwvfoENh4B5fk0IWC6U

To create the above dataframe:


    


In [ ]:
conda install -n myenv -c conda-forge geopy --yes


In [ ]:
conda install -n myenv -c conda-forge beautifulsoup4 --yes


In [ ]:
conda install -n myenv -c conda-forge lxml --yes

In [1]:
CLIENT_ID = 'C3NLZY0MJGCJENLKP4I5R0NONFIS3A4DPRNLGV33MWAU5Y3O' # your Foursquare ID
CLIENT_SECRET = '2OOM4WNGBAPZ5M0NC0E4JF31EZXS2KD2XQETJVHAIGTOWMZM' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails are hidden as CLIENT_ID, CLIENT_SECRET.')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails are hidden as CLIENT_ID, CLIENT_SECRET.


The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
--Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated 
with a comma as shown in row 11 in the above table.

--If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.


In [2]:
from bs4 import BeautifulSoup
import requests

source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

#with open("List of postal codes of Canada  M - Wikipedia.html", encoding='utf8') as html:
#    soup = BeautifulSoup(html)
soup = BeautifulSoup(source)
#print (soup)
t = soup.table
table_data = []
t_headers = ['PostalCode', 'Borough', 'Neighborhood']

for tr in t.tbody.find_all("tr"): # find all tr's from table's tbody
    t_value = []
    # Each table row is stored in the form of
    #t_row = {'Postal code', 'Borough', 'Neighborhood(s)'}
    # find all td's(3) in tr and zip it with t_header
    #for td, th in zip(tr.find_all("td"), t_headers): 
    #    t_row[th] = td.text.replace('\n', '').strip()
    for td in tr.find_all('td'):
        t_value.append(td.text.replace('\n', '').strip())
    if t_value:
        table_data.append(t_value)

mydata = table_data
#print (mydata[0:10])
cleaned = {v[0]: v for v in mydata if v and v[1]!='Not assigned' }
#print (cleaned)
for k in cleaned:
    cleaned[k][2] = cleaned[k][2].replace(' / ', ', ')
    if cleaned[k][2] == 'Not assigned': cleaned[k][2] = cleaned[k][1]

for a in list(cleaned.keys())[0:10]:
     print (cleaned[a])



['M3A', 'North York', 'Parkwoods']
['M4A', 'North York', 'Victoria Village']
['M5A', 'Downtown Toronto', 'Regent Park, Harbourfront']
['M6A', 'North York', 'Lawrence Manor, Lawrence Heights']
['M7A', 'Downtown Toronto', "Queen's Park, Ontario Provincial Government"]
['M9A', 'Etobicoke', 'Islington Avenue']
['M1B', 'Scarborough', 'Malvern, Rouge']
['M3B', 'North York', 'Don Mills']
['M4B', 'East York', 'Parkview Hill, Woodbine Gardens']
['M5B', 'Downtown Toronto', 'Garden District, Ryerson']


In [3]:
import pandas as pd

t_headers = ['PostalCode', 'Borough', 'Neighborhood']
data = pd.DataFrame.from_dict(cleaned, orient='index', columns=t_headers)

data.head()


,PostalCode,Borough,Neighborhood
M3A,M3A,North York,Parkwoods
M4A,M4A,North York,Victoria Village
M5A,M5A,Downtown Toronto,"Regent Park, Harbourfront"
M6A,M6A,North York,"Lawrence Manor, Lawrence Heights"
M7A,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [4]:
data.shape

(103, 3)

In [5]:
conda install -n myenv -c conda-forge geocoder --yes


Solving environment: ...working... done

Note: you may need to restart the kernel to use updated packages.
# All requested packages already installed.




In [6]:
import geocoder # import geocoder

for postal_code in cleaned:
  # initialize your variable to None
  lat_lng_coords = None
  ctr = 0
  # loop until you get the coordinates
  while(lat_lng_coords is None):
    ctr +=1
    g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
    lat_lng_coords = g.latlng
  #print ("Got {} after {} tries".format(postal_code, ctr))

  latitude = lat_lng_coords[0]
  longitude = lat_lng_coords[1]
  cleaned[postal_code].append(latitude)
  cleaned[postal_code].append(longitude)


t_headers = ['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']
data2 = pd.DataFrame.from_dict(cleaned, orient='index', columns=t_headers)

In [7]:
data2.head()


,PostalCode,Borough,Neighborhood,Latitude,Longitude
M3A,M3A,North York,Parkwoods,43.753259,-79.329656
M4A,M4A,North York,Victoria Village,43.725882,-79.315572
M5A,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
M6A,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
M7A,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [ ]:
conda install -n myenv -c conda-forge folium --yes

In [162]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering librar
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import numpy as np # library to handle data in a vectorized manner
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from IPython.display import HTML, display

A map showing a blue dot for each neighborhood.

In [164]:
latitude = 43.6532
longitude = -79.3832
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11, control_scale=True)

# add markers to map
for lat, lng, borough, neighborhood in zip(data2['Latitude'], data2['Longitude'], data2['Borough'], data2['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
#map_toronto



display(HTML(map_toronto._repr_html_()))

We will only look at neighborhoods that have Toronto in the name of the Borough

In [38]:
#toronto_subset = data2['Toronto' in data2['Borough'] ].reset_index(drop=True)
toronto_subset = data2[data2['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_subset.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [152]:
toronto_subset.shape

(39, 5)

There are 39 neighborhoods in boroughs that have Toronto in their name. The map shows this subset of neighborhoods.

In [155]:
address="Toronto, CA"
geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [165]:
# create map of Toronto using latitude and longitude values
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=13, control_scale=True)

# add markers to map
for lat, lng, label in zip(toronto_subset['Latitude'], toronto_subset['Longitude'], toronto_subset['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
    
#toronto_map
display(HTML(toronto_map._repr_html_()))

### Venues in the Neightborhood that Foursquare has.
#### Radius
I used the suggested radius of 500 meters.

In [13]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


## 2. Explore Neighborhoods in Toronto
First define a function to get venues for the neighborhood and run it on each neighborhood.

In [121]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

toronto_venues = getNearbyVenues(names=toronto_subset['Neighborhood'],
                                   latitudes=toronto_subset['Latitude'],
                                   longitudes=toronto_subset['Longitude'], radius=500
                                  )


Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town,

In [122]:
toronto_venues.groupby('Neighborhood').count()

print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))


There are 231 unique categories.


### Analyze each neighborhood
convert each venue type to a column

In [123]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

print (len(toronto_onehot.columns))
# move neighborhood column to the first column
#fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
fixed_columns = list(toronto_onehot.columns)
fixed_columns.pop(fixed_columns.index('Neighborhood'))
fixed_columns.insert(0, 'Neighborhood')
print (len(fixed_columns))

toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

231
231


,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [128]:
toronto_onehot.shape


(1623, 231)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [129]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Berczy Park,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.017544,0.0,0.0,0.0,0.0,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.043478
2,Business reply mail Processing Centre,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.062500
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.0625,0.0625,0.0625,0.125,0.125,0.125,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
4,Central Bay Street,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.015625,0.0,0.0,0.0,0.0,0.015625


In [130]:
toronto_grouped.shape

(39, 231)

In [127]:
num_top_venues =8

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')


----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1        Cocktail Bar  0.05
2            Beer Bar  0.04
3                Café  0.04
4              Bakery  0.04
5         Cheese Shop  0.04
6  Seafood Restaurant  0.04
7          Restaurant  0.04


----Brockton, Parkdale Village, Exhibition Place----
                   venue  freq
0                   Café  0.13
1            Coffee Shop  0.09
2         Breakfast Spot  0.09
3            Yoga Studio  0.04
4     Italian Restaurant  0.04
5              Pet Store  0.04
6  Performing Arts Venue  0.04
7              Nightclub  0.04


----Business reply mail Processing Centre----
            venue  freq
0     Yoga Studio  0.06
1   Garden Center  0.06
2      Comic Shop  0.06
3     Pizza Place  0.06
4      Restaurant  0.06
5         Butcher  0.06
6   Burrito Place  0.06
7  Farmers Market  0.06


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
             

Put the results in a dataframe

First a function to sort the venues in descending order (most frequent first)

In [131]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()
#neighborhoods_venues_sorted.shape

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Café,Bakery,Beer Bar,Cheese Shop,Restaurant,Seafood Restaurant,Concert Hall,Basketball Stadium
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Bakery,Stadium,Burrito Place,Restaurant,Climbing Gym,Pet Store,Performing Arts Venue
2,Business reply mail Processing Centre,Yoga Studio,Auto Workshop,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Light Rail Station,Comic Shop,Pizza Place,Butcher
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Sculpture Garden,Boutique,Plane,Rental Car Location,Harbor / Marina,Boat or Ferry,Bar
4,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Burger Joint,Salad Place,Ice Cream Shop,Dessert Shop,Thai Restaurant,Japanese Restaurant


In [132]:
neighborhoods_venues_sorted.shape

(39, 11)

## Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [133]:
# set number of clusters
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
#toronto_grouped_clustering.head()

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
print ("First 10 lables: {}".format( kmeans.labels_[0:10] ))


First 10 lables: [2 2 2 2 2 0 2 2 2 2]


In [100]:
# add clustering labels
neighborhoods_venues_sorted.drop('Cluster Labels', 1, inplace=True)
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_subset

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head(6) # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Park,Restaurant,Theater,Pub,Café,Italian Restaurant,Bakery,Breakfast Spot,Performing Arts Venue
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Sushi Restaurant,Park,Japanese Restaurant,Gastropub,Restaurant,Ice Cream Shop
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Coffee Shop,Japanese Restaurant,Gastropub,Hotel,Café,Ramen Restaurant,Middle Eastern Restaurant,Sushi Restaurant,Park,Plaza
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Café,Coffee Shop,Seafood Restaurant,Gastropub,Theater,Italian Restaurant,Cosmetics Shop,Restaurant,American Restaurant,Bakery
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Pub,Coffee Shop,Sandwich Place,Bank,Breakfast Spot,Bar,Japanese Restaurant,Chocolate Shop,Tea Room,Bagel Shop
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Café,Hotel,Japanese Restaurant,Restaurant,Cocktail Bar,Bakery,Beer Bar,Gastropub,Grocery Store


In [134]:
#toronto_merged.dtypes
toronto_merged.shape
#print(toronto_merged['Cluster Labels'])

(39, 16)

We have some clusters with nan values, (ex: the last row above) so need to drop them

In [102]:
toronto_merged.dropna(subset=['Cluster Labels'], inplace = True)
toronto_merged.shape

(39, 16)

we had 103 rows, but had to drop 3 with nan.  Now visualize the  clusters.


In [166]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12, control_scale=True)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for neigh, lat, lon, poi, cluster in zip(toronto_merged['Neighborhood'], toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], pd.Series(toronto_merged['Cluster Labels'], dtype=pd.Int64Dtype())):
    cluster = int(cluster)
    #print ( "{} Cluster {} is {}".format(neigh, cluster, type(cluster)))
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
#map_clusters
display(HTML(map_clusters._repr_html_())) 

In [ ]:
pd.Series(toronto_merged['Cluster Labels'], dtype=pd.Int64Dtype())

In [136]:
cluster0 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]
cluster0.head(100)

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park, Harbourfront",0,Coffee Shop,Park,Restaurant,Theater,Pub,Café,Italian Restaurant,Bakery,Breakfast Spot,Performing Arts Venue
1,"Queen's Park, Ontario Provincial Government",0,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Sushi Restaurant,Park,Japanese Restaurant,Gastropub,Restaurant,Ice Cream Shop
2,"Garden District, Ryerson",0,Coffee Shop,Japanese Restaurant,Gastropub,Hotel,Café,Ramen Restaurant,Middle Eastern Restaurant,Sushi Restaurant,Park,Plaza
3,St. James Town,0,Café,Coffee Shop,Seafood Restaurant,Gastropub,Theater,Italian Restaurant,Cosmetics Shop,Restaurant,American Restaurant,Bakery
4,The Beaches,0,Pub,Coffee Shop,Sandwich Place,Bank,Breakfast Spot,Bar,Japanese Restaurant,Chocolate Shop,Tea Room,Bagel Shop
5,Berczy Park,0,Coffee Shop,Café,Hotel,Japanese Restaurant,Restaurant,Cocktail Bar,Bakery,Beer Bar,Gastropub,Grocery Store
6,Central Bay Street,0,Coffee Shop,Café,Japanese Restaurant,Art Gallery,Clothing Store,Italian Restaurant,Ramen Restaurant,Ice Cream Shop,Grocery Store,Sushi Restaurant
7,Christie,0,Café,Grocery Store,Park,Diner,Coffee Shop,Italian Restaurant,Indian Restaurant,Hardware Store,Restaurant,Candy Store
8,"Richmond, Adelaide, King",0,Café,Coffee Shop,Hotel,Theater,Restaurant,Clothing Store,Japanese Restaurant,Sushi Restaurant,Concert Hall,Cosmetics Shop
9,"Dufferin, Dovercourt Village",0,Coffee Shop,Gym,Bakery,Smoke Shop,Portuguese Restaurant,Pharmacy,Bar,Park,Italian Restaurant,Diner


Cluster 0 has a lot of coffee shops, cafes and restaurants.

In [137]:
cluster1 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]
cluster1.head(100)

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Lawrence Park,1,Coffee Shop,Swim School,Bus Line,Park,Yoga Studio,Eastern European Restaurant,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant


Cluster 1 has a lot of coffee shops, Swim schools and restaurants.

In [139]:
cluster2 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]
cluster2.head(100)

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Roselawn,2,Home Service,Playground,Pet Store,Music Venue,Ice Cream Shop,Garden,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant


In [ ]:
Cluster 2 has a lot of Home services, playgrounds and pet stores.

In [140]:
cluster3 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]
cluster3.head(100)

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Forest Hill North & West,3,Jewelry Store,Park,Sushi Restaurant,Trail,Dry Cleaner,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant


In [ ]:
Cluster 3 has a lot of jewelry stores and parks, sushi restaurants, and trails.

In [141]:
cluster4 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]
cluster4.head(100)

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
33,Rosedale,4,Park,Trail,Candy Store,Playground,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner


In [ ]:
Cluster 4 has a lot of parks and trails